In [31]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


pd.set_option('display.max_columns', None)

In [49]:
# Importing data
df_movies = pd.read_csv('tmdb_5000_movies.csv')
print('Shape of tmdb dataset',df_movies.shape)
df_credits = pd.read_csv('tmdb_5000_credits.csv')
print('Shape of tmdb dataset',df_credits.shape)

Shape of tmdb dataset (4803, 20)
Shape of tmdb dataset (4803, 4)


In [50]:
df_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [66]:
# Merging te data sets
df = pd.merge(df_movies, df_credits, on = 'title')
print("The shape of the data is",df.shape)
df.head(1)

The shape of the data is (4809, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [95]:
# Selecting the features from dataset
movies = df.loc[:,['movie_id','title','overview','genres','keywords','cast','crew']]

In [96]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [97]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [98]:
movies.dropna(inplace=True)

In [99]:
import ast

def convert(text):
    """
    This function inputs text from df and output the clean text with relevent data
    """
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name']) 
    return l


def convert2(text):
    """
    This function is used to get the 3 names
    """
    l = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter<=3:            
            l.append(i['name']) 
        counter += 1
    return l


def get_directer(text):
    """
    This function gets the director names.
    """
    l = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
    return l



In [103]:
# Example how ast work
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [100]:
movies['genres'] = movies.genres.apply(convert)
movies['keywords'] = movies.keywords.apply(convert)
movies['cast'] = movies.cast.apply(convert2)
movies['crew'] = movies.crew.apply(get_directer)


In [90]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]


In [105]:
movies.overview[0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [107]:
# Converting to a list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [108]:
# Creating a master feature
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [113]:
new_df = movies.drop(['overview', 'genres', 'keywords', 'cast', 'crew' ], axis = 1)

In [116]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

In [117]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [119]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 6000, stop_words = 'english')

In [120]:
vector  = cv.fit_transform(new_df['tags']).toarray()
vector.shape

(4806, 6000)

In [122]:
from sklearn.metrics.pairwise import cosine_similarity
cosim = cosine_similarity(vector)

In [128]:
# Gives us the index of te movie title
new_df[new_df['title']== 'The Lego Movie'].index[0]

744

In [144]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(cosim[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)
        
def recommend_n(movie,n):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(cosim[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:n+1]:
        print(new_df.iloc[i[0]].title)

In [151]:
recommend('Lost in Space')

Silent Running
Space Pirate Captain Harlock
Spaceballs
Moonraker
Gravity


In [150]:
recommend_n('Lost in Space', 10)

Silent Running
Space Pirate Captain Harlock
Spaceballs
Moonraker
Gravity
Mission to Mars
2001: A Space Odyssey
Avatar
Space Dogs
Lockout


In [147]:
# Saving new_df and cosine similarity matrix
import pickle

pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(cosim,open('co_sim.pkl','wb'))